In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip install transformers==4.28.1 datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from transformers import BertTokenizer, BertModel, AdamW, BertConfig
from transformers.models.bert.modeling_bert import BertEmbeddings
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
import spacy
import copy

from transformers.utils import logging
logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
nlp = spacy.load("en_core_web_sm")

spacy_dep2id = {dep: i+2 for i, dep in enumerate(nlp.get_pipe("parser").labels)}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/wac_project/QA_v2/df_sentences_context.xlsx')
df["text"] = df["text"].fillna("").astype(str)
df["context"] = df["context"].fillna("").astype(str)

label_list = list(df['label'].unique())

label2id = {label: i for i, label in enumerate(sorted(set(label_list)))}
id2label = {i: label for label, i in label2id.items()}

df['label'] = df['label'].map(label2id)

In [7]:
def balance_df(df):
  assert set(df['y'].unique()).issubset({0, 1})
  target_ratio = 0.15
  ones_df = df[df['y'] == 1]
  zeros_df = df[df['y'] == 0]
  n_ones = len(ones_df)
  total_desired = int(n_ones / target_ratio)
  n_zeros_needed = total_desired - n_ones
  sampled_zeros_df = zeros_df.sample(n=n_zeros_needed, random_state=42) # Downsample 0s
  balanced_df = pd.concat([ones_df, sampled_zeros_df]).sample(frac=1, random_state=42).reset_index(drop=True) # Combine
  return balanced_df

def perfect_balancing_downsampling(df, y='label'):
  min_count = min(df[y].value_counts())
  labels = df[y].unique()
  df_list = []
  for label in labels:
    df_label = df[df[y] == label]
    df_label = df_label.sample(n=min_count, random_state=42)
    df_list.append(df_label)
  df_final = pd.concat(df_list).reset_index(drop=True).copy(deep=True)
  return df_final

def add_pos_tags(text):
    doc = nlp(text)
    return " ".join([f"{token.text}/{token.pos_}" for token in doc])

def replace_with_pos_tags(text):
    doc = nlp(text)
    return " ".join([f"{token.pos_}" for token in doc])

import spacy
from transformers import BertTokenizerFast

# Load models
nlp = spacy.load("en_core_web_sm")
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

def map_tokens(text):
    doc = nlp(text)
    spacy_tokens = list(doc)

    # Step 1: Build a char->spaCy-token-index map
    char_to_spacy = {}
    for i, token in enumerate(spacy_tokens):
        for pos in range(token.idx, token.idx + len(token)):
            char_to_spacy[pos] = i

    # Step 2: BERT tokenize with offsets
    encoding = bert_tokenizer(text, return_offsets_mapping=True, return_special_tokens_mask=True)
    offsets = encoding['offset_mapping']
    input_ids = encoding['input_ids']
    tokens = bert_tokenizer.convert_ids_to_tokens(input_ids)
    special_mask = encoding['special_tokens_mask']

    # Step 3: Align
    aligned = []
    for token, (start, end), is_special in zip(tokens, offsets, special_mask):
        if is_special or start == end:
            continue  # Skip [CLS], [SEP], padding, etc.

        # Use the first character position of the BERT token to find matching spaCy token
        spa_idx = char_to_spacy.get(start, None)
        if spa_idx is not None:
            aligned.append(spacy_tokens[spa_idx])

    return aligned

def get_dep_ids(text):
    mapped_tokens = map_tokens(text)
    dep_ids = [0] + [spacy_dep2id[token.dep_] for token in mapped_tokens] + [1]
    return dep_ids

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
add_pos = False
encode_pair = False
embed_pos_tags = True

In [9]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/wac_project/QA_v2/df_sentences_context.xlsx')
df["text"] = df["text"].fillna("").astype(str)
df["context"] = df["context"].fillna("").astype(str)

label_list = list(df['label'].unique())

label2id = {label: i for i, label in enumerate(sorted(set(label_list)))}
id2label = {i: label for label, i in label2id.items()}

df['label'] = df['label'].map(label2id)

if add_pos:
  print('Add POS Tags - ', add_pos)
  df['text'] = df['text'].apply(replace_with_pos_tags)
  # df['context'] = df['context'].apply(add_pos_tags)

train_id, temp_id = train_test_split(
    df['file_id'].unique(),
    test_size=0.3,
    random_state=42
)

val_id, test_id = train_test_split(
    temp_id,
    test_size=0.5,
    random_state=42
)

train_df = df[df['file_id'].isin(train_id)].copy(deep=True)
val_df = df[df['file_id'].isin(val_id)].copy(deep=True)
test_df = df[df['file_id'].isin(test_id)].copy(deep=True)

# train_df = perfect_balancing_downsampling(train_df)

def encode_df(df_subset, encode_pair=False):
    # Ensure all values are strings
    df_subset = df_subset.copy()
    df_subset["text"] = df_subset["text"].fillna("").astype(str)
    df_subset["context"] = df_subset["context"].fillna("").astype(str)

    if encode_pair:
        ret = tokenizer(
            list(df_subset["text"]),
            list(df_subset["context"]),
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt",
            return_overflowing_tokens=False)
    else:
        ret = tokenizer(
            list(df_subset["text"]),
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt",
            return_overflowing_tokens=False)

    return ret

print('Sentence Pair Encoding - ', encode_pair)

train_encodings = encode_df(train_df, encode_pair)
val_encodings = encode_df(val_df, encode_pair)
test_encodings = encode_df(test_df, encode_pair)

train_labels = torch.tensor(train_df["label"].astype(int).values).to(device)
val_labels = torch.tensor(val_df["label"].astype(int).values).to(device)
test_labels = torch.tensor(test_df["label"].astype(int).values).to(device)

train_inputs = {k: v.to(device) for k, v in train_encodings.items()}
val_inputs = {k: v.to(device) for k, v in val_encodings.items()}
test_inputs = {k: v.to(device) for k, v in test_encodings.items()}

Sentence Pair Encoding -  False


In [10]:
if embed_pos_tags:
  all_dep_ids = []
  for text in train_df['text']:
    dep_ids = get_dep_ids(text)
    all_dep_ids.append(torch.tensor(dep_ids))
  train_inputs['dep_ids'] = pad_sequence(all_dep_ids, batch_first=True, padding_value=0).to(device)

  all_dep_ids = []
  for text in val_df['text']:
    dep_ids = get_dep_ids(text)
    all_dep_ids.append(torch.tensor(dep_ids))
  val_inputs['dep_ids'] = pad_sequence(all_dep_ids, batch_first=True, padding_value=0).to(device)

  all_dep_ids = []
  for text in test_df['text']:
    dep_ids = get_dep_ids(text)
    all_dep_ids.append(torch.tensor(dep_ids))
  test_inputs['dep_ids'] = pad_sequence(all_dep_ids, batch_first=True, padding_value=0).to(device)

In [11]:
# class BertMultiClassClassifier(nn.Module):
#     def __init__(self, num_classes):
#         super().__init__()
#         self.bert = BertModel.from_pretrained("bert-base-cased")
#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

#     def forward(self, input_ids, attention_mask, token_type_ids):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         cls_output = outputs.pooler_output
#         x = self.dropout(cls_output)
#         return self.classifier(x)

In [12]:
# class CustomEmbeddings(BertEmbeddings):
#     def __init__(self, config, num_dep_tags):
#         super().__init__(config)
#         self.dep_embeddings = nn.Embedding(num_dep_tags, config.hidden_size)

#     def forward(self, input_ids, token_type_ids, position_ids, dep_ids):
#         base = super().forward(input_ids, token_type_ids, position_ids)
#         # if dep_ids is not None:
#         dep_embed = self.dep_embeddings(dep_ids)
#         base = base + dep_embed
#         return base

In [13]:
# class CustomBertMultiClassClassifier(BertMultiClassClassifier):
#     def __init__(self, num_classes, num_dep_tags):
#         super().__init__(num_classes)
#         self.embeddings = CustomEmbeddings(self.bert.config, num_dep_tags)

#     def forward(self, input_ids, attention_mask, token_type_ids, dep_ids):
#         print('forward')
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, dep_ids=dep_ids)
#         print('After outputs')
#         cls_output = outputs.pooler_output
#         x = self.dropout(cls_output)
#         return self.classifier(x)

In [14]:
import torch
import torch.nn as nn
from transformers import BertPreTrainedModel, BertConfig
from transformers.models.bert.modeling_bert import (
    BertModel,
    BertEmbeddings,
    BertEncoder,
    BertPooler
)

# Step 1: Custom Embeddings with dependency tags
class CustomEmbeddings(BertEmbeddings):
    def __init__(self, config, num_dep_tags):
        super().__init__(config)
        self.dep_embeddings = nn.Embedding(num_dep_tags, config.hidden_size)

    def forward(self, input_ids=None, token_type_ids=None, position_ids=None, dep_ids=None):
        embeddings = super().forward(input_ids, token_type_ids, position_ids)
        if dep_ids is not None:
            dep_embeddings = self.dep_embeddings(dep_ids)
            embeddings = embeddings * dep_embeddings
        return embeddings

# Step 2: Custom BERT model (not calling super().forward)
class CustomBertModel(BertPreTrainedModel):
    def __init__(self, config, num_dep_tags):
        super().__init__(config)
        self.embeddings = CustomEmbeddings(config, num_dep_tags)
        self.encoder = BertEncoder(config)
        self.pooler = BertPooler(config)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, dep_ids=None):
        # 1. Embedding layer
        embedding_output = self.embeddings(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            dep_ids=dep_ids
        )

        # 2. Extend attention mask
        extended_attention_mask = attention_mask[:, None, None, :]
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        # 3. Encoder
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=[None] * self.config.num_hidden_layers,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True
        )

        # 4. Pooling
        sequence_output = encoder_outputs.last_hidden_state
        pooled_output = self.pooler(sequence_output)

        return {
            "last_hidden_state": sequence_output,
            "pooler_output": pooled_output
        }

# Step 3: Final classifier using CustomBertModel
class CustomBertMultiClassClassifier(nn.Module):
    def __init__(self, num_classes, num_dep_tags):
        super().__init__()
        config = BertConfig.from_pretrained("bert-base-cased")
        self.bert = CustomBertModel(config=config, num_dep_tags=num_dep_tags)
        self.bert.load_state_dict(
            BertModel.from_pretrained("bert-base-cased").state_dict(), strict=False
        )
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids, dep_ids):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            dep_ids=dep_ids
        )
        cls_output = outputs["pooler_output"]
        x = self.dropout(cls_output)
        return self.classifier(x)


In [15]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [16]:
from sklearn.utils.class_weight import compute_class_weight

num_labels = train_df['label'].unique()
num_labels.sort()

class_weights = compute_class_weight(class_weight='balanced', classes=num_labels, y=train_df['label'])

In [17]:
num_labels = list(label2id.values())
string_labels = [id2label[i] for i in num_labels]
num_classes = len(label2id)
n_run_metrics = []

for run in range(5):
  print(f'Run {run+1}')
  model = CustomBertMultiClassClassifier(num_classes, len(spacy_dep2id)+3).to(device)

  optimizer = AdamW(model.parameters(), lr=2e-5)
  criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(class_weights).float().to(device)) # weight=torch.from_numpy(class_weights).float().to(device)

  batch_size = 32
  epochs = 50
  patience = 3
  best_val_loss = float("inf")
  patience_counter = 0
  best_model_state = None

  model.train()

  for epoch in range(epochs):
      total_train_loss = 0
      model.train()

      for i in range(0, len(train_labels), batch_size):
          input_batch = {k: v[i:i+batch_size] for k, v in train_inputs.items()}
          label_batch = train_labels[i:i+batch_size]

          optimizer.zero_grad()
          logits = model(**input_batch)
          loss = criterion(logits, label_batch)
          loss.backward()
          optimizer.step()

          total_train_loss += loss.item()

      avg_train_loss = total_train_loss / (len(train_labels) // batch_size + 1)

      torch.cuda.empty_cache()
      torch.cuda.ipc_collect()

      model.eval()
      val_loss = 0

      with torch.no_grad():
          for i in range(0, len(val_labels), batch_size):
              batch = {k: v[i:i+batch_size] for k, v in val_inputs.items()}
              label_batch = val_labels[i:i+batch_size]

              logits = model(**batch)
              loss = criterion(logits, label_batch)

              val_loss += loss.item()

      avg_val_loss = val_loss / (len(val_labels) // batch_size + 1)
      print(f"Epoch {epoch+1}/{epochs} — Train Loss: {avg_train_loss:.4f} — Val Loss: {avg_val_loss:.4f}")


      if val_loss < best_val_loss:
          best_val_loss = val_loss
          patience_counter = 0
          best_model_state = copy.deepcopy(model.state_dict())
      else:
          patience_counter += 1
          if patience_counter >= patience:
              print("Early stopping triggered.")
              break

      torch.cuda.empty_cache()
      torch.cuda.ipc_collect()

  if best_model_state is not None:
      model.load_state_dict(best_model_state)

  model.eval()
  all_preds = []
  all_true = []

  with torch.no_grad():
      for i in range(0, len(test_labels), batch_size):
          batch_inputs = {k: v[i:i+batch_size] for k, v in test_inputs.items()}
          label_batch = test_labels[i:i+batch_size]

          logits = model(**batch_inputs)#.logits
          preds = torch.argmax(logits, dim=1)

          all_preds.extend(preds.cpu().numpy())
          all_true.extend(label_batch.cpu().numpy())

  torch.cuda.empty_cache()
  torch.cuda.ipc_collect()

  from sklearn.metrics import precision_recall_fscore_support
  import pandas as pd
  import numpy as np

  y_true = np.array(all_true)
  y_pred = np.array(all_preds)

  current_run_metrics = np.asarray(precision_recall_fscore_support(y_true, y_pred, labels=num_labels, zero_division=0))
  n_run_metrics.append(current_run_metrics)

  print('Evaluation Results:')
  print(pd.DataFrame(current_run_metrics.T, columns=['precision', 'recall', 'F1', 'support'], index=string_labels))

avg_n_run_metrics = pd.DataFrame(np.mean(np.asarray(n_run_metrics), axis=0).T, columns=['precision','recall','F1','support'], index=string_labels)
print('\nAverge Metrics')
print(avg_n_run_metrics)

Run 1


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Epoch 1/50 — Train Loss: 1.9922 — Val Loss: 1.9165
Epoch 2/50 — Train Loss: 1.9717 — Val Loss: 1.9248
Epoch 3/50 — Train Loss: 1.9532 — Val Loss: 1.9147
Epoch 4/50 — Train Loss: 1.9531 — Val Loss: 1.9149
Epoch 5/50 — Train Loss: 1.9449 — Val Loss: 1.9164
Epoch 6/50 — Train Loss: 1.9441 — Val Loss: 1.9105
Epoch 7/50 — Train Loss: 1.9408 — Val Loss: 1.9119


KeyboardInterrupt: 

In [ ]:
print('Train', train_df['label'].value_counts(), '\n')
print('Eval', val_df['label'].value_counts(), '\n')
print('Test', test_df['label'].value_counts(), '\n')

In [ ]:
def common_strings(series1, series2):
    set1 = set(series1)
    set2 = set(series2)
    return list(set1.intersection(set2))

print('Train x Eval - ', common_strings(train_id, val_id))
print('Train x Test - ', common_strings(train_id, test_id))
print('Test x Eval - ', common_strings(test_id, val_id))

In [ ]:
# from torch.nn.functional import softmax

# model.eval()

# with torch.no_grad():
#     logits = model(**test_inputs)
#     probs = softmax(logits, dim=1)
#     preds = torch.argmax(probs, dim=1)

# test_df["predicted_label"] = preds.cpu().numpy()
# test_df["confidence"] = probs.max(dim=1).values.cpu().numpy()

# test_df.to_excel("/content/drive/MyDrive/Colab Notebooks/wac_project/QA_v2/test_results_inference.xlsx", index=False)